In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import json
import random

### Data Preparation

In [2]:
def get_dataset(directory):
    feature = []
    label = []
    for foldername in os.listdir(directory):
        folder = os.path.join(directory, foldername)
        if os.path.isdir(folder):
            files = os.listdir(folder)
            for filename in files:
                rel_path = os.path.join(directory, foldername, filename)
                temp_label = filename.split('.')[0].split('_')[0]
                if 'a' in temp_label:
                    label = np.append(label,'alcoholic')
                else:
                    label = np.append(label,'control')
                
                df_data = np.loadtxt(rel_path, delimiter=",")
                feature.append(df_data.T)
    return np.array(feature), np.array(label)

In [14]:
def get_batch(path):
    # loading extracted feature & label
    x, y = get_dataset(path)
    y = pd.DataFrame(y)
    
    # Encode the labels
    label_map = {"alcoholic": 1, "control": 0}
    y[0] = y[0].map(label_map)
    
    dataset = tf.data.Dataset.from_tensor_slices((x, y[0]))
    dataset = dataset.shuffle(len(y[0])).batch(32)

    return dataset

### Model Definition

In [20]:
def create_model():
    model = keras.models.Sequential()

    model.add(layers.Input(shape=(366,)))
    model.add(layers.Reshape((366, 1)))

    model.add(layers.Conv1D(filters=16, kernel_size=4, activation="relu"))
    model.add(layers.MaxPooling1D(pool_size=2))

    model.add(layers.Conv1D(filters=8, kernel_size=2, activation="relu"))
    model.add(layers.MaxPooling1D(pool_size=2))

    model.add(layers.Flatten())

    model.add(layers.Dense(2160, activation="relu"))

    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(1080, activation="relu"))

    model.add(layers.Dropout(0.2))

    model.add(layers.Dense(1, activation="sigmoid"))

    return model

### Main Program

In [21]:
train = get_batch('../Features/smni_cmi_test_feature_256')

train_size = int(len(list(train.as_numpy_iterator()))*0.8)

train_ds = train.take(train_size)
val_ds = train.skip(train_size)


In [19]:
for x in val_ds:
    print(x)

(<tf.Tensor: shape=(32, 366), dtype=float64, numpy=
array([[7.06712722e+03, 7.84701434e+00, 2.87291905e+02, ...,
        2.93989709e+00, 1.23154756e+01, 9.87707358e+00],
       [1.57220802e+03, 8.31488066e-01, 3.35143489e+01, ...,
        1.27694243e+01, 4.56108024e+01, 7.93976020e+01],
       [7.08962869e+02, 1.94986418e-01, 6.80302927e+00, ...,
        8.83749177e-01, 1.64077697e+00, 3.55619653e+00],
       ...,
       [2.57953899e+04, 7.08163742e-01, 3.01469481e+01, ...,
        5.01447841e+00, 1.95511996e+01, 1.12378881e+01],
       [6.51052433e+04, 3.89568136e+00, 1.28917402e+02, ...,
        8.70762447e+00, 2.30449212e+01, 3.04236546e+01],
       [4.55321244e+04, 2.14535841e+00, 1.09387208e+02, ...,
        7.82359311e+00, 1.08454381e+01, 3.77817210e+01]])>, <tf.Tensor: shape=(32,), dtype=int64, numpy=
array([0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0], dtype=int64)>)
(<tf.Tensor: shape=(32, 366), dtype=float64, numpy=
ar

In [22]:
model = create_model()
model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(train_ds, epochs=200, validation_data=(val_ds))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_6 (Reshape)         (None, 366, 1)            0         
                                                                 
 conv1d_12 (Conv1D)          (None, 363, 16)           80        
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 181, 16)          0         
 g1D)                                                            
                                                                 
 conv1d_13 (Conv1D)          (None, 180, 8)            264       
                                                                 
 max_pooling1d_13 (MaxPoolin  (None, 90, 8)            0         
 g1D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 720)              